In [2]:
import sys
import os
import django
from pathlib import Path

# Manually set the root directory and the settings module
root_dir = "C:/Users/USER/Documents/recommender/src"
settings_module = "cfehome.settings"

os.environ.setdefault("DJANGO_SETTINGS_MODULE", settings_module)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Add the root directory to sys.path
if root_dir not in sys.path:
    sys.path.insert(0, root_dir)
    print(f"'{root_dir}' successfully added to sys.path")

# Setup Django
try:
    django.setup()
    print("Django setup successfully")
except Exception as e:
    print(f"Error setting up Django: {e}")

'C:/Users/USER/Documents/recommender/src' successfully added to sys.path
Django setup successfully


In [4]:
import numpy as np
import pandas as pd
from django.contrib.auth import get_user_model
from django.db.models import F, Count, Sum, IntegerField
from movies.models import Movie
from ratings.models import Rating
import random

User = get_user_model()

In [6]:
rating_obj = Rating.objects.filter(user__username='Sydney', value__gte=5).first()
rating_obj.content_object

<Movie: The Poseidon Adventure>

In [7]:
movie_id = rating_obj.content_object.id
value = rating_obj.value
user_id = rating_obj.user_id

In [8]:
print(f"Movie {movie_id} was rated {value} by {user_id}")

Movie 551 was rated 5 by 2


In [11]:
other_user_ids = Rating.objects.filter(
        object_id=rating_obj.object_id,
        content_type=rating_obj.content_type,
        value__gte=rating_obj.value
).exclude(user=rating_obj.user).values_list('user_id', flat=True)



highly_rated = Rating.objects.filter(user_id__in=other_user_ids, value__gte=4)

In [12]:
rec_users = []
rec_movies = []
for rating_obj in highly_rated:
    if rating_obj.user not in rec_users:
        rec_users.append(rating_obj.user)
    object_id = rating_obj.object_id
    if object_id not in rec_movies:
        rec_movies.append(object_id)

In [13]:
print(len(rec_users), len(set(rec_movies)))

24 2339


In [14]:
import numpy as np


In [16]:
#a list of 2 movie attributes
movie_1 = ['sci-fi', 'comedy', 'Stars latest action movie star']
movie_2 = ['historical fiction', 'drama', 'Stars latest action movie star']

#a list of user preferences
user_a = ['comedy', 'sci-fi', 'Stars latest action movie star']
print(f"User has {len(user_a)} primary preferences")

#total number of potential features
num_unique_features = len(set(movie_1 + movie_2))
print(f"These movies have {num_unique_features} unique features.")

User has 3 primary preferences
These movies have 5 unique features.


In [17]:
my_preferences = np.array([.98, .85, .75])

In [18]:
movie_attribute_scores = np.array([.96, .99, .92])

movie_2_attribute_scores = np.array([-.93, -.45, .5])

In [19]:
(my_preferences * movie_attribute_scores).sum()

2.4723

In [20]:
(my_preferences * movie_2_attribute_scores).sum()

-0.9189

In [22]:
max_score = (np.array([1,1,1]) * np.array([1,1,1])).sum()
min_score = (np.array([1,1,1]) * np.array([-1,-1,-1])).sum()
print(max_score, min_score)

3 -3


In [26]:
ids = Movie.objects.all().popular()[:15]
most_active_users = Rating.objects.all().annotate(
    user_count=Count("user")
).order_by('-user_count').values_list('user_id')[:5_000]

ratings_qs = Rating.objects.filter(
    object_id__in=ids,
    user_id__in=most_active_users,
    active=True, value__gte=0
).annotate(movie=F("object_id")).values('user', 'value', 'movie')

df = pd.DataFrame(ratings_qs)

In [27]:
df.pivot_table(index='user', columns=['movie'], values='value', fill_value='')[:15]

movie,110,260,296,318,356,457,480,527,588,593,608,858,2762,2959,4993
user,,,,,,,,,,,,,,,
2,4.0,,4.0,,3.0,3.0,4.0,4.0,3.0,3.0,,,,,
3,4.0,,5.0,5.0,5.0,,,3.0,3.0,3.0,,,4.0,5.0,
4,,5.0,5.0,,5.0,,5.0,,5.0,,,5.0,,,
5,,,,,4.0,,,,4.0,,,3.0,4.0,,
7,5.0,5.0,,5.0,3.0,,4.0,,4.0,,,,,,
8,4.0,4.0,4.0,5.0,4.0,5.0,,5.0,,5.0,,5.0,5.0,4.0,4.0
9,,,,4.0,,,,5.0,,4.0,5.0,,4.0,,
10,,,,4.0,,,,,,,,,,,
11,,,5.0,,,,,,,,,,3.0,,


In [28]:
movie_1 = np.array([1, 1, 1, 1])
user_1 = np.array([1, 1, 1, 1])
(user_1 * movie_1).sum()

4